In [18]:
from pyproj import Proj, transform
from pyproj import Transformer
from pyproj import CRS
from pyparsing import Word, alphas, ZeroOrMore, Suppress, Optional
import os

In [2]:
crs_utm = CRS.from_user_input(28407)

In [3]:
crs_utm

<Derived Projected CRS: EPSG:28407>
Name: Pulkovo 1942 / Gauss-Kruger zone 7
Axis Info [cartesian]:
- X[north]: Northing (metre)
- Y[east]: Easting (metre)
Area of Use:
- name: Georgia - onshore west of 36°E; Russian Federation - onshore 36°E to 42°E; Ukraine - onshore east of 36°E.
- bounds: (36.0, 41.43, 42.0, 69.23)
Coordinate Operation:
- name: 6-degree Gauss-Kruger zone 7
- method: Transverse Mercator
Datum: Pulkovo 1942
- Ellipsoid: Krassowsky 1940
- Prime Meridian: Greenwich

In [4]:
crs_utm = CRS.from_user_input(4284)
crs_utm

<Geographic 2D CRS: EPSG:4284>
Name: Pulkovo 1942
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: Armenia; Azerbaijan; Belarus; Estonia - onshore; Georgia - onshore; Kazakhstan; Kyrgyzstan; Latvia - onshore; Lithuania - onshore; Moldova; Russian Federation - onshore; Tajikistan; Turkmenistan; Ukraine - onshore; Uzbekistan.
- bounds: (19.57, 35.14, -168.97, 81.91)
Datum: Pulkovo 1942
- Ellipsoid: Krassowsky 1940
- Prime Meridian: Greenwich

In [5]:
#x_in, y_in = 40, 35 #latitude (широта), longitude(долгота)

In [102]:
x_in, y_in = 40, 36 #latitude (широта), longitude(долгота)
transformer = Transformer.from_crs('epsg:4284', 'epsg:28407')
X, Y = transformer.transform(x_in, y_in)
print(f'X={X}, Y={Y}')

X=4433921.003568104, Y=7243793.583852794


# Пересчёт координат береговой линии

In [73]:
f_1 = open('Shore_line.wpt', 'r+')
f_2 = open('Shore_line_proj.txt', 'w+')
f_2.write("lat, lon, X, Y\n")
l = f_1.readlines()
#print(l[4])
for s in range(4, len(l)):
    for i in range(len(l[s].split(","))):
        lat = l[s].replace(" ", "").split(",")[2]
        lon = l[s].replace(" ", "").split(",")[3]
        #print(l[4].replace(" ", "").split(",")[i])
    #print(f'lat={l[4].replace(" ", "").split(",")[2]}, lon={l[4].replace(" ", "").split(",")[3]}')
    X, Y = transformer.transform(lat, lon)
    #print(f'X={X}, Y={Y}')
    #f_2.write(f", X={X}, Y={Y}\n")
    f_2.write(f"{lat}, {lon}, {X}, {Y}\n")
    #print(l[4].split(", "))
f_1.close()
f_2.close()

# Точки для гравиметрических наблюдений

In [89]:
f = open('Gravity_points.wpt', 'r+')
l = f.readlines()
l_s = l[4].split(",")
print(l_s)
l_s_j = ",".join(l_s)
print(l_s_j)
f.close()

['1', '1', '  40.546710', '  41.204681', '44835.0618482', '  0', ' 0', ' 3', '         0', '     65535', '', ' 0', ' 0', '    0', '   -777', ' 6', ' 0', '17', '0', '10.0', '2', '', '', '\n']
True


In [93]:
with open('Gravity_points.wpt', "r") as f:
    lat = l[4].replace(" ", "").split(",")[2]
    lon = l[4].replace(" ", "").split(",")[3]
    lines = f.readlines()
with open('Gravity_points.wpt', "w") as f:
    for line in lines:
        if line != lines[4]:
            f.write(line)

In [105]:
with open('Gravity_points.wpt', "r") as f:
    lat = l[4].replace(" ", "").split(",")[2]
    lon = l[4].replace(" ", "").split(",")[3]
    lines = f.readlines()
with open('Gravity_points.wpt', "w") as f:
    for line in lines:
        if line != lines[4]:
            f.write(line)
f = open('Gravity_points.wpt', 'r+')
print(lat)
print(lon)
transformer = Transformer.from_crs('epsg:4284', 'epsg:28407')
X_c, Y_c = transformer.transform(lat, lon) #ещё старый трансформер
print(X_c)
print(Y_c)
X_lc, Y_lc = X_c+2500, Y_c-2500
transformer = Transformer.from_crs('epsg:28407', 'epsg:4284') #обратный трансформер
#lat, lon = transformer.transform(X_lc, Y_lc)
f.write("OziExplorer Waypoint File Version 1.1\nWGS 84\nReserved 2\n\n")
#f.write(f"WP,D,1,{lat},{lon},10/01/2022,22:00:00,A,N,-9999\n")
t=1
for i in range(6):
    for j in range(6):
        #X_lc, Y_lc = X - 1000*j, Y + 1000*i # уточнить шаг по профилю
        lat, lon = transformer.transform(X_lc-1000*j, Y_lc+1000*i)
        #f.write(f"WP,D,1,{lat},{lon},10/01/2022,22:00:00,A,N,-9999\n")
        f.write(f"{t},{t},  {lat},  {lon},44835.0618482,  0, 0, 3,         0,     65535,, 0, 0,    0,   -777, 6, 0,17,0,10.0,2,,,\n")
        #f.write(f"{t},{t},{lat},{lon},44835.0618482,0,0,3,0,65535,,0,0,0,-777,6,0,17,0,10.0,2,,,\n")
        t = t+1
        
f.close()

40.546710
41.204681
4492651.544423742
7686766.549271134


In [103]:
x_in, y_in = 4433921.003568104, 7243793.583852794 #latitude (широта), longitude(долгота)
transformer = Transformer.from_crs('epsg:28407', 'epsg:4284')
lat , lon = transformer.transform(x_in, y_in)
print(f'lat={round(lat)}, lon={round(lon)}')

lat=40, lon=36
